In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

In [ ]:
# https://gist.github.com/perrygeo/4512375
def scale_linear_bycolumn(rawpoints, high=1.0, low=0.0):
    mins = np.min(rawpoints, axis=0)
    maxs = np.max(rawpoints, axis=0)
    rng = maxs - mins
    return high - (((high - low) * (maxs - rawpoints)) / rng)

In [ ]:
df = pd.read_csv('faithful.dat', delim_whitespace=True, skiprows=25)
df['eruptions'] = scale_linear_bycolumn(df['eruptions'])
df['waiting'] = scale_linear_bycolumn(df['waiting'])
df.plot(kind='scatter', x='eruptions', y='waiting')

In [ ]:
from scipy.stats import multivariate_normal

def gmm_mixture(df, mu1=None, cov1=None, mu2=None, cov2=None, p_mix=None):
    df = df[['eruptions', 'waiting']]
    # Take initial guesses for mean vectors, covariance matrices,
    # and mixing proportion
    if mu1 is None: mu1 = np.random.random(2)
    if cov1 is None: cov1 = np.random.random() * np.eye(2)
    if mu2 is None: mu2 = np.random.random(2)
    if cov2 is None: cov2 = np.random.random() * np.eye(2)
    if p_mix is None: p_mix = np.random.random()

    # E-step: Compute responsibilities
    var1 = multivariate_normal(mean=mu1, cov=cov1)
    var2 = multivariate_normal(mean=mu2, cov=cov2)
    resp = p_mix*var2.pdf(df) / ((1-p_mix)*var1.pdf(df) + p_mix*var2.pdf(df))
    resp = np.array([resp])

    mu1_arr = np.array([mu1])
    mu2_arr = np.array([mu2])

    diff = 1
    epsilon = 0.0001
    while diff > epsilon:
        # M-step: compute weighted means and variances
        mu1 = sum((1-resp).dot(df)) / sum(sum(1-resp))
        cov1 = np.diag(sum((1-resp).dot(np.square(df-mu1))) / sum(sum(1-resp)))
        if np.count_nonzero(np.less(cov1, epsilon * np.eye(2))) > 0:
            cov1 = np.random.random() * np.eye(2)

        mu2 = sum(resp.dot(df)) / sum(sum(resp))
        cov2 = np.diag(sum(resp.dot(np.square(df-mu2))) / sum(sum(resp)))
        if np.count_nonzero(np.less(cov2, epsilon * np.eye(2))) > 0:
            cov2 = np.random.random() * np.eye(2)
        
        mu1_arr = np.append(mu1_arr, [mu1], axis=0)
        mu2_arr = np.append(mu2_arr, [mu2], axis=0)

        old_p_mix = p_mix
        p_mix = sum(sum(resp)) / len(df)
        
        # E-step: Compute responsibilities
        var1 = multivariate_normal(mean=mu1, cov=cov1)
        var2 = multivariate_normal(mean=mu2, cov=cov2)
        resp = p_mix*var2.pdf(df) / ((1-p_mix)*var1.pdf(df) + p_mix*var2.pdf(df))
        resp = np.array([resp])
        diff = abs(p_mix - old_p_mix)
    return mu1_arr, mu2_arr

In [ ]:
trajectories = []
for i in range(50):
    trajectories.append(gmm_mixture(df))

# http://stackoverflow.com/questions/8945699/gnuplot-linecolor-variable-in-matplotlib/18516488#18516488
fig = plt.figure()

for i in trajectories[0]:
    print i[-1]
    points = np.array(i).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments, cmap=plt.get_cmap('Spectral'),
                        norm=plt.Normalize(1, len(i)))
    lc.set_array(np.arange(1,len(i)+1))
    lc.set_linewidth(2)

    plt.gca().add_collection(lc)

axcb = fig.colorbar(lc)
axcb.set_label('iterations')

plt.show()
pd.DataFrame([len(trajectory[0]) for trajectory in trajectories], columns=['iterations']).hist()

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2)

In [ ]:
from sklearn.covariance import empirical_covariance

trajectories = []
for i in range(50):
    df['label'] = kmeans.fit_predict(df[['eruptions', 'waiting']])
    mix = gmm_mixture(df,
                      mu1=kmeans.cluster_centers_[0][0:2],
                      cov1=empirical_covariance(df[df['label']==0][['eruptions', 'waiting']]), 
                      mu2=kmeans.cluster_centers_[1][0:2],
                      cov2=empirical_covariance(df[df['label']==1][['eruptions', 'waiting']]), 
                      p_mix = float(np.count_nonzero(df['label']))/len(df['label']))
    trajectories.append(mix)

# http://stackoverflow.com/questions/8945699/gnuplot-linecolor-variable-in-matplotlib/18516488#18516488
fig = plt.figure()

for i in trajectories[0]:
    print i[-1]
    points = np.array(i).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments, cmap=plt.get_cmap('Spectral'),
                        norm=plt.Normalize(1, len(i)))
    lc.set_array(np.arange(1,len(i)+1))
    lc.set_linewidth(2)

    plt.gca().add_collection(lc)

axcb = fig.colorbar(lc)
axcb.set_label('iterations')

plt.show()
pd.DataFrame([len(trajectory[0]) for trajectory in trajectories], columns=['iterations']).hist()